In [11]:
import os
from math import gcd
from functools import reduce
from dotenv import load_dotenv
from aocd import get_data

In [2]:
load_dotenv()
secret_key = os.getenv("SECRET_KEY")
os.environ['AOC_SESSION'] = secret_key

data = get_data(year=2023, day=8)

## Part 1
One of the camel's pouches is labeled "maps" - sure enough, it's full of documents (your puzzle input) about how to navigate the desert. At least, you're pretty sure that's what they are; one of the documents contains a list of left/right instructions, and the rest of the documents seem to describe some kind of network of labeled nodes.

It seems like you're meant to use the left/right instructions to navigate the network. Perhaps if you have the camel follow the same instructions, you can escape the haunted wasteland!

After examining the maps for a bit, two nodes stick out: AAA and ZZZ. You feel like AAA is where you are now, and you have to follow the left/right instructions until you reach ZZZ.

This format defines each node of the network individually. For example:
```
RL

AAA = (BBB, CCC)
BBB = (DDD, EEE)
CCC = (ZZZ, GGG)
DDD = (DDD, DDD)
EEE = (EEE, EEE)
GGG = (GGG, GGG)
ZZZ = (ZZZ, ZZZ)
```
Starting with AAA, you need to look up the next element based on the next left/right instruction in your input. In this example, start with AAA and go right (R) by choosing the right element of AAA, CCC. Then, L means to choose the left element of CCC, ZZZ. By following the left/right instructions, you reach ZZZ in 2 steps.

Of course, you might not find ZZZ right away. If you run out of left/right instructions, repeat the whole sequence of instructions as necessary: RL really means RLRLRLRLRLRLRLRL... and so on. For example, here is a situation that takes 6 steps to reach ZZZ:
```
LLR

AAA = (BBB, BBB)
BBB = (AAA, ZZZ)
ZZZ = (ZZZ, ZZZ)
```
Starting at AAA, follow the left/right instructions. How many steps are required to reach ZZZ?

In [5]:
def parse_input(data):
    instructions = [x for x in data.splitlines()[0]]
    nodes = {}
    for line in data.splitlines()[2:]:
        node, element_data = line.split(' = ')
        elements = (element_data[1:4], element_data[-4:-1])
        nodes[node] = elements

    return instructions, nodes

def find_zzz(instructions, nodes):
    max_steps = 50000
    steps = 0
    start_node = 'AAA'
    end_node = None
    
    while end_node != 'ZZZ':
        for d in instructions:
            steps += 1
            if steps >= max_steps: return steps
            i = 0 if d == 'L' else 1
            end_node = nodes[start_node][i]
            start_node = end_node
            if end_node == 'ZZZ': break
    
    return steps

def solve_part1(data):
    instructions, nodes = parse_input(data)
    steps = find_zzz(instructions, nodes)
    
    return steps

In [6]:
solve_part1(data)

13771

## Part 2
After examining the maps a bit longer, your attention is drawn to a curious fact: the number of nodes with names ending in A is equal to the number ending in Z! If you were a ghost, you'd probably just start at every node that ends with A and follow all of the paths at the same time until they all simultaneously end up at nodes that end with Z.

For example:
```
LR

11A = (11B, XXX)
11B = (XXX, 11Z)
11Z = (11B, XXX)
22A = (22B, XXX)
22B = (22C, 22C)
22C = (22Z, 22Z)
22Z = (22B, 22B)
XXX = (XXX, XXX)
```
Here, there are two starting nodes, 11A and 22A (because they both end with A). As you follow each left/right instruction, use that instruction to simultaneously navigate away from both nodes you're currently on. Repeat this process until all of the nodes you're currently on end with Z. (If only some of the nodes you're on end with Z, they act like any other node and you continue as normal.) In this example, you would proceed as follows:
```
Step 0: You are at 11A and 22A.
Step 1: You choose all of the left paths, leading you to 11B and 22B.
Step 2: You choose all of the right paths, leading you to 11Z and 22C.
Step 3: You choose all of the left paths, leading you to 11B and 22Z.
Step 4: You choose all of the right paths, leading you to 11Z and 22B.
Step 5: You choose all of the left paths, leading you to 11B and 22C.
Step 6: You choose all of the right paths, leading you to 11Z and 22Z.
```
So, in this example, you end up entirely on nodes that end in Z after 6 steps.

Simultaneously start on every node that ends with A. How many steps does it take before you're only on nodes that end with Z?

In [12]:
def find_z_to_z(instructions, nodes):
    start_nodes = [n for n in nodes if n.endswith('A')]
    z_steps = []

    for start_node in start_nodes:
        max_steps = 100000
        steps = 0
        end_node = 'aaa'
        while not end_node.endswith('Z'):
            for d in instructions:
                steps += 1
                if steps >= max_steps:
                    return steps
                i = 0 if d == 'L' else 1
                end_node = nodes[start_node][i]
                start_node = end_node
                if end_node.endswith('Z'):
                    z_steps.append(steps)
                    break

    return z_steps

def lcm(a, b):
    return abs(a * b) // gcd(a, b)

def lcm_multiple(numbers):
    return reduce(lcm, numbers)

def solve_part2(data):
    instructions, nodes = parse_input(data)
    z_steps = find_z_to_z(instructions, nodes)
    answer = lcm_multiple(z_steps)

    return answer

In [13]:
solve_part2(data)

13129439557681